<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv3/blob/main/4_adverserial_attacks_ipynb_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
download_links = ['https://github.com/mostafa-ja/mal_adv3/raw/main/data/X_redefined_sparse_matrix.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/data/DNN_params%20.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/data/labels.pt',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/data/vocab.pkl',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/data/adverserial_attacks_functions.py',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/data/best_model%20_RFGSM.pth',
]

In [2]:
import gdown
output_filepath = '/content/'
for link in download_links:
  gdown.download(link, output_filepath)


Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/data/X_redefined_sparse_matrix.npz
To: /content/X_redefined_sparse_matrix.npz
100%|██████████| 2.31M/2.31M [00:00<00:00, 9.41MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/data/DNN_params%20.pth
To: /content/DNN_params%20.pth
100%|██████████| 8.17M/8.17M [00:00<00:00, 60.1MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/data/labels.pt
To: /content/labels.pt
100%|██████████| 517k/517k [00:00<00:00, 6.12MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/data/vocab.pkl
To: /content/vocab.pkl
100%|██████████| 9.18M/9.18M [00:00<00:00, 17.5MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/data/adverserial_attacks_functions.py
To: /content/adverserial_attacks_functions.py
45.5kB [00:00, 50.9MB/s]                   
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/data/best_model%20_RFGSM.pth
To: /conten

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from scipy import sparse
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch.nn.functional as F
import random
import time

from adverserial_attacks_functions import *

torch.manual_seed(0)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
# Load the dictionary from the file
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

for i, (key, value) in enumerate(vocab.items()):
    print((key, value))
    if i >= 5:
        break

('android/media/mediaplayer->start', 141045)
('android/app/activity->setcontentview', 140900)
('android/os/vibrator->cancel', 141093)
('android.permission.vibrate', 140720)
('android.hardware.touchscreen', 137091)
('android.intent.action.main', 138335)


Memory-Efficient but slow when we want to convert back to tensor by
.to_dense().to(torch.float32)

```
from scipy import sparse
from sklearn.model_selection import train_test_split

# Load dataset
X_redefined = sparse.load_npz("X_redefined_sparse_matrix.npz")
labels_tensor = torch.load('labels.pt')

# Split data into train, validation, and test sets with stratified sampling
X_train_val, X_test, labels_train_val, labels_test = train_test_split(X_redefined, labels_tensor, test_size=0.2, stratify=labels_tensor, random_state=42)
X_train, X_val, labels_train, labels_val = train_test_split(X_train_val, labels_train_val, test_size=0.2, stratify=labels_train_val, random_state=42)

# Create PyTorch sparse tensors directly from the sparse matrices
train_dataset = TensorDataset(torch.sparse_coo_tensor(torch.tensor(X_train.nonzero()), torch.tensor(X_train.data), X_train.shape), labels_train)
val_dataset = TensorDataset(torch.sparse_coo_tensor(torch.tensor(X_val.nonzero()), torch.tensor(X_val.data), X_val.shape), labels_val)
test_dataset = TensorDataset(torch.sparse_coo_tensor(torch.tensor(X_test.nonzero()), torch.tensor(X_test.data), X_test.shape), labels_test)

# Clear unnecessary variables
del X_redefined, labels_tensor, X_train_val, X_test, labels_train_val, labels_test, X_train, X_val, labels_train, labels_val
```



In [5]:
# Load dataset
X_redefined = sparse.load_npz("X_redefined_sparse_matrix.npz")
labels_tensor = torch.load('labels.pt')

# Split data into train, validation, and test sets with stratified sampling
X_train_val, X_test, labels_train_val, labels_test = train_test_split(X_redefined, labels_tensor, test_size=0.2, stratify=labels_tensor, random_state=42)
X_train, X_val, labels_train, labels_val = train_test_split(X_train_val, labels_train_val, test_size=0.2, stratify=labels_train_val, random_state=42)

# Combine features and labels into datasets
# we use dtype=torch.int8, for Memory-Efficient here, later we will convert to float
train_dataset = TensorDataset(torch.tensor(X_train.toarray(), dtype=torch.int8), labels_train)
val_dataset = TensorDataset(torch.tensor(X_val.toarray(), dtype=torch.int8), labels_val)
test_dataset = TensorDataset(torch.tensor(X_test.toarray(), dtype=torch.int8), labels_test)

# Clear unnecessary variables
del X_redefined, labels_tensor, X_train_val, X_test, labels_train_val, labels_test, X_train, X_val, labels_train, labels_val

In [6]:
# Define the DataLoader for training, validation, and test sets
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
class MalwareDetectionModel(nn.Module):
    def __init__(self, input_size=10000, hidden_1_size=200, hidden_2_size=200, num_labels=2, dropout_prob=0.6):
        super(MalwareDetectionModel, self).__init__()

        self.input_size = input_size
        self.hidden_1_size = hidden_1_size
        self.hidden_2_size = hidden_2_size
        self.num_labels = num_labels
        self.dropout_prob = dropout_prob

        self.fc1 = nn.Linear(input_size, hidden_1_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(hidden_1_size, hidden_2_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(hidden_2_size, num_labels)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.log_softmax(x)
        return x


In [8]:
model_DNN = MalwareDetectionModel().to(device)
# Load model parameters
model_DNN.load_state_dict(torch.load('DNN_params%20.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [9]:
def adversarial_training(model, train_loader, val_loader, attack, adv_epochs=10, lr=0.001, weight_decay=0., device=device, verbose=True, **kwargs):
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    total_time = 0.
    nbatches = len(train_loader)
    best_acc_val = 0.
    acc_val_adv_be = 0.
    best_epoch = 0

    for epoch in range(adv_epochs):
        epoch_losses = []
        epoch_accuracies = []

        for idx_batch, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(torch.float32).to(device), y_batch.to(device)
            batch_size = x_batch.shape[0]

            # Separate malicious and benign samples
            mal_x_batch, ben_x_batch = x_batch[y_batch.squeeze() == 1], x_batch[y_batch.squeeze() == 0]
            mal_y_batch, ben_y_batch = y_batch[y_batch.squeeze() == 1], y_batch[y_batch.squeeze() == 0]

            # Generate adversarial examples
            model.eval()
            pertb_mal_x = attack(mal_x_batch, mal_y_batch, model, **kwargs)
            x_batch = torch.cat([ben_x_batch, pertb_mal_x], dim=0)
            y_batch = torch.cat([ben_y_batch, mal_y_batch])
            model.train()

            # Forward pass and backward pass
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss_train = criterion(outputs, y_batch.view(-1).long())
            loss_train.backward()
            optimizer.step()

            # Calculate metrics
            epoch_losses.append(loss_train.item())
            _, predicted = torch.topk(outputs, k=1)
            acc_train = (predicted == y_batch).sum().item() / len(y_batch)
            epoch_accuracies.append(acc_train)

            # Print batch level information
            if verbose:
                print(f'Mini batch: {idx_batch + 1}/{nbatches} | Epoch: {epoch + 1}/{adv_epochs} | Batch Loss: {loss_train.item():.4f} | Batch Accuracy: {acc_train * 100:.2f}%')

        # Calculate epoch level metrics
        mean_loss = np.mean(epoch_losses)
        mean_accuracy = np.mean(epoch_accuracies) * 100

        # Print epoch level information
        if verbose:
            print(f'Training loss (epoch level): {mean_loss:.4f} | Train accuracy: {mean_accuracy:.2f}%')

        # Evaluation on validation set
        model.eval()
        avg_acc_ad_val = []
        avg_acc_val = []
        for x_val, y_val in val_loader:
            x_val, y_val = x_val.to(device), y_val.to(device)
            outputs = model(x_val)
            _, predicted = torch.topk(outputs, k=1)
            acc_val = (predicted == y_val).sum().item() / len(y_val)
            avg_acc_val.append(acc_val)

            # Generate adversarial examples for validation set
            mal_x_batch, mal_y_batch = x_val[y_val.squeeze() == 1], y_val[y_val.squeeze() == 1]
            pertb_mal_x = attack(mal_x_batch, mal_y_batch, model)
            outputs = model(pertb_mal_x)
            _, y_pred = torch.topk(outputs, k=1)

            acc_ad_val = (y_pred == 1.).sum().item() / len(y_pred)
            avg_acc_ad_val.append(acc_ad_val)

        # Calculate validation accuracy
        assert len(avg_acc_ad_val) > 0
        acc_all = (np.mean(avg_acc_val) + np.mean(avg_acc_ad_val)) / 2.

        # Update best validation accuracy
        if acc_all >= best_acc_val:
            best_acc_val = acc_all
            acc_val_adv_be = np.mean(avg_acc_ad_val)
            best_epoch = epoch + 1
            torch.save(model.state_dict(), 'best_model.pth')

        # Print validation results
        if verbose:
            print(f"\tVal accuracy(without attack) {np.mean(avg_acc_val) * 100:.4}% and accuracy(with attack) {np.mean(avg_acc_ad_val) * 100:.4}% under attack and overall accuracy {acc_all * 100:.4}%.")
            print(f"\tModel select at epoch {best_epoch} with validation accuracy {best_acc_val * 100:.4}% and accuracy {acc_val_adv_be * 100:.4}% under attack.")


In [10]:
# AT-rFGSM: Adversarial Taraining based on rFGSM attack
#model_AT = MalwareDetectionModel().to(device)

#attack_param = {"k":50, "epsilon":0.02, 'random':True, "is_sample":False, 'is_report_loss_diff':True}
#adversarial_training(model_AD, train_loader, val_loader, adv_epochs=50, attack=dfgsm_k, **attack_param)

In [21]:
def adv_predict(test_loader, model, attack, device, **kwargs):

    if attack == mimicry:
      # Pre-select benign samples
      benign_samples = []
      for x_batch, y_batch in test_loader:
        benign_samples.append(x_batch[y_batch.squeeze() == 0])

      ben_x = torch.cat(benign_samples, dim=0).to(device)
      del benign_samples

    model.eval()
    avg_acc_ad_test = []
    avg_acc_test = []
    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test, y_test = x_test.to(torch.float32).to(device), y_test.to(device)
            outputs = model(x_test)
            _, predicted = torch.topk(outputs, k=1)
            acc_test = (predicted == y_test).sum().item() / len(y_test)
            avg_acc_test.append(acc_test)

            # Generate adversarial examples for test set
            mal_x_batch, mal_y_batch = x_test[y_test.squeeze() == 1], y_test[y_test.squeeze() == 1]

            if attack == mimicry:
                pertb_mal_x = mimicry(ben_x, mal_x_batch, model, **kwargs)
            else :
                with torch.enable_grad():
                    pertb_mal_x = attack(mal_x_batch, mal_y_batch, model, **kwargs)

            outputs = model(pertb_mal_x)
            _, y_pred = torch.topk(outputs, k=1)

            acc_ad_test = (y_pred == 1.).sum().item() / len(y_pred)
            avg_acc_ad_test.append(acc_ad_test)

    # Calculate test accuracy
    print(f"Adversarial accuracy (without attack): {np.mean(avg_acc_test) * 100:.4}% | Under attack: {np.mean(avg_acc_ad_test) * 100:.4}%.")
    if attack == mimicry:
      del ben_x


In [13]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('best_model%20_RFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [ ]:
adv_predict(test_loader, model_DNN, attack=pgd, is_report_loss_diff=False, device=device)

Adversarial accuracy (without attack): 99.34% | Under attack: 0.0%.


In [22]:
adv_predict(test_loader, model_AT_rFGSM, attack=pgd, is_report_loss_diff=False, device=device)

Adversarial accuracy (without attack): 99.36% | Under attack: 90.83%.


In [ ]:
# Groose
attack_params = {"k":100, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, grosse_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, grosse_k, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 0.0%.
Adversarial accuracy (without attack): 99.36% | Under attack: 83.93%.


In [ ]:
# BCA
attack_params = {"k":100, 'is_report_loss_diff':False, 'use_sample':False}
adv_predict(test_loader, model_DNN, bca_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, bca_k, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 0.0%.
Adversarial accuracy (without attack): 99.36% | Under attack: 83.93%.


In [ ]:
# BGA
attack_params = {"k":100, 'is_report_loss_diff':False, 'use_sample':False}
adv_predict(test_loader, model_DNN, bga_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, bga_k, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 0.0%.
Adversarial accuracy (without attack): 99.36% | Under attack: 90.87%.


In [ ]:
# rFGSM
attack_params = {"k":100, "epsilon":0.02, 'random':True, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, dfgsm_k, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 0.0%.
Adversarial accuracy (without attack): 99.36% | Under attack: 89.91%.


In [ ]:
# PGD-l1
attack_params = {"k":500, "step_length":1., 'norm':'l1', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 0.0%.
Adversarial accuracy (without attack): 99.36% | Under attack: 82.9%.


In [ ]:
# PGD-l2
attack_params = {"k":200, "step_length":0.05, 'norm':'l2', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 40.03%.
Adversarial accuracy (without attack): 99.36% | Under attack: 91.56%.


In [ ]:
# PGD-linf
attack_params = {"k":500, "step_length":0.002, 'norm':'linf', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 0.0%.
Adversarial accuracy (without attack): 99.36% | Under attack: 86.74%.


In [ ]:
# Mimicry×1
attack_params = {"trials":1, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 84.44%.
Adversarial accuracy (without attack): 99.36% | Under attack: 92.63%.


In [ ]:
# Mimicry×10
attack_params = {"trials":10, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 4.929%.
Adversarial accuracy (without attack): 99.36% | Under attack: 83.69%.


In [ ]:
# Mimicry×30
attack_params = {"trials":30, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 4.929%.
Adversarial accuracy (without attack): 99.36% | Under attack: 83.69%.


In [ ]:
# Mimicry×100
attack_params = {"trials":100, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 4.929%.
Adversarial accuracy (without attack): 99.36% | Under attack: 83.59%.


In [ ]:
# Mimicry×1000
attack_params = {"trials":1000, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 1.517%.
Adversarial accuracy (without attack): 99.36% | Under attack: 82.68%.


In [ ]:
# Mimicry×1000
attack_params = {"trials":5000, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)

Adversarial accuracy (without attack): 99.34% | Under attack: 0.0%.
Adversarial accuracy (without attack): 99.36% | Under attack: 74.55%.


In [ ]:
attack_params = {'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.001}, "steps":500}
adv_predict(test_loader, model_AT_rFGSM, StepwiseMax_onestep2, device, **attack_params)

step-wise max: attack effectiveness 0.000%.
step-wise max: attack effectiveness 10.000%.


KeyboardInterrupt: 

In [ ]:
# Define different attacks with their parameters
attacks = [
    (dfgsm_k, {"k":100, "epsilon":0.02, 'is_report_loss_diff' : False}),
    (bga_k, {"k":100, 'is_report_loss_diff' : False}),
    (bca_k, {"k":100, 'is_report_loss_diff' : False}),
    (grosse_k, {"k":100, 'is_report_loss_diff' : False}),

    (pgd, {'k': 100, 'step_length': 1., 'norm': 'l1', 'is_report_loss_diff' : False}),
    (pgd, {'k': 200, 'step_length': 0.05, 'norm': 'l2', 'is_report_loss_diff' : False}),
    (pgd, {'k': 100, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : False}),
    (StepwiseMax_onestep2, {'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.0008}, "steps":650}),

    # Add more attacks as needed
]

# Iterate over each attack and its parameters
for attack_func, attack_params in attacks:
    print(f"Running attack: {attack_func.__name__} with parameters: {attack_params}")
    adv_predict(test_loader, model_AT_rFGSM, attack_func, device, **attack_params)
    print()  # Print an empty line for separation


Running attack: dfgsm_k with parameters: {'k': 50, 'epsilon': 0.02, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 83.43%.

Running attack: bga_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 91.57%.

Running attack: bca_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.93%.

Running attack: grosse_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.93%.

Running attack: pgd with parameters: {'k': 100, 'step_length': 1.0, 'norm': 'l1', 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.24%.

Running attack: pgd with parameters: {'k': 200, 'step_length': 0.05, 'norm': 'l2', 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 91.67%.

Running att



```
def mimic_attack_effectiveness_optimized(test_loader, model, seed, trials=1000, device="cuda:0"):
  """
  Calculates the effectiveness of the mimic attack on the given model.

  Args:
      test_loader: A PyTorch dataloader containing the test data.
      model: The PyTorch model to be attacked.
      seed: The random seed for reproducibility.
      trials: The number of random samples to use from the benign class (default: 1000).
      device: The device to use for computations (default: "cuda:0" if available, otherwise "cpu").

  Returns:
      The effectiveness of the mimic attack as a percentage (float).
  """

  torch.manual_seed(seed)
  model.eval()

  # Initialize counters
  successful_attacks = 0
  total_malicious_samples = 0

  # Pre-select benign samples for efficiency
  benign_samples = []
  for x_batch, y_batch in test_loader:
    benign_samples.append(x_batch[y_batch.squeeze() == 0])

  ben_x = torch.cat(benign_samples, dim=0).to(device)

  # Clear unnecessary variables
  del benign_samples

  trials = min(trials, len(ben_x))


  for x_batch, y_batch in test_loader:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    malicious_samples = x_batch[y_batch.squeeze() == 1]

    if len(malicious_samples) > 0:
      # Expand dimensions for efficient broadcasting
      malicious_samples_expanded = malicious_samples.unsqueeze(1).expand(-1, trials, -1)

      # Generate random indices outside the loop
      seed += 1
      torch.manual_seed(seed)
      indices = torch.randperm(len(ben_x), device=device)[:trials]
      trial_vectors_expanded = ben_x[indices].unsqueeze(0)

      # Perform the mimic attack and update counters
      modified_x = torch.clamp(malicious_samples_expanded + trial_vectors_expanded, min=0., max=1.)
      _, done = get_loss(modified_x.view(-1, modified_x.shape[-1]), torch.ones(trials * malicious_samples.shape[0], 1, device=device), model)
      successful_attacks += (done.view(malicious_samples.shape[0], trials).sum(dim=1) > 0).sum().item()
      total_malicious_samples += malicious_samples.shape[0]

  # Calculate and print attack effectiveness
  attack_effectiveness = (successful_attacks / total_malicious_samples) * 100 if total_malicious_samples > 0 else 0
  print(f"Mimic attack effectiveness: {attack_effectiveness:.3f}%.")

  return attack_effectiveness  # Added return statement for clarity

```



In [ ]:
def mimicry(ben_x, malicious_samples, model_DNN, trials=30, seed=230, is_report_loss_diff=False):
    """
    Perform a mimicry attack.

    Args:
    - ben_x (torch.Tensor): Benign samples tensor.
    - malicious_samples (torch.Tensor): Malicious samples tensor.
    - model_DNN (torch.nn.Module): PyTorch model used for the attack.
    - trials (int): Number of trials for the attack.
    - seed (int): Random seed for reproducibility.
    - is_report_loss_diff (bool): Flag to indicate whether to report attack effectiveness.

    Returns:
    - adv_x (torch.Tensor): Adversarial examples tensor.
    """

    # Ensure trials do not exceed the length of ben_x
    trials = min(trials, len(ben_x))

    # Get the number of malicious samples
    n_samples = len(malicious_samples)

    if n_samples > 0:
        # Expand dimensions for efficient broadcasting
        malicious_samples_expanded = malicious_samples.unsqueeze(1).expand(-1, trials, -1)

        # Generate random indices for sampling from ben_x
        torch.manual_seed(seed)
        indices = torch.randperm(len(ben_x), device=ben_x.device)[:trials]
        trial_vectors_expanded = ben_x[indices].unsqueeze(0)

        # Perform the mimic attack
        pertbx = torch.clamp(malicious_samples_expanded + trial_vectors_expanded, min=0., max=1.)

        # Compute the loss and check if adversarial examples are successful
        loss, done = get_loss(pertbx.view(-1, pertbx.shape[-1]), torch.ones(n_samples * trials, 1, device=ben_x.device), model_DNN)

        # Add maximum loss to successful attacks to differentiate
        max_v = loss.max()
        loss[done] += max_v

        # Reshape the loss and done tensors
        loss = loss.view(n_samples, trials)
        done = done.view(n_samples, trials)

        # Report attack effectiveness if required
        if is_report_loss_diff:
            n_done = torch.any(done, dim=-1).sum()
            print(f"Mimicry*{trials}: Attack effectiveness {n_done / n_samples * 100:.3f}%.")

        # Get the index of the maximum loss for each sample
        _, indices = loss.max(dim=-1)
        adv_x = pertbx[torch.arange(n_samples), indices]

        del pertbx, loss, done, malicious_samples_expanded, trial_vectors_expanded

        return adv_x
    else:
        print("No malicious samples found.")
        return None




In [ ]:
# Mimicry×30
attack_params = {"trials":10, 'is_report_loss_diff':True}
#adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)

Mimicry*10: Attack effectiveness 0.000%.
Mimicry*10: Attack effectiveness 0.000%.
Mimicry*10: Attack effectiveness 33.333%.
Mimicry*10: Attack effectiveness 0.000%.
Mimicry*10: Attack effectiveness 10.526%.
Mimicry*10: Attack effectiveness 40.000%.
Mimicry*10: Attack effectiveness 37.500%.
Mimicry*10: Attack effectiveness 18.750%.
Mimicry*10: Attack effectiveness 12.500%.
Mimicry*10: Attack effectiveness 7.692%.
Mimicry*10: Attack effectiveness 23.529%.
Mimicry*10: Attack effectiveness 30.769%.
Mimicry*10: Attack effectiveness 21.429%.
Mimicry*10: Attack effectiveness 20.000%.
Mimicry*10: Attack effectiveness 0.000%.
Mimicry*10: Attack effectiveness 17.647%.
Mimicry*10: Attack effectiveness 36.842%.
Mimicry*10: Attack effectiveness 8.333%.
Mimicry*10: Attack effectiveness 12.500%.
Mimicry*10: Attack effectiveness 20.000%.
Mimicry*10: Attack effectiveness 22.222%.
Mimicry*10: Attack effectiveness 23.077%.
Mimicry*10: Attack effectiveness 11.111%.
Mimicry*10: Attack effectiveness 33.333%

In [ ]:
for x_test, y_test in test_loader:

        # Generate adversarial examples for test set
        mal_x_batch, mal_y_batch = x_test[y_test.squeeze() == 1], y_test[y_test.squeeze() == 1]

        pertb_mal_x = mimicry(ben_x, mal_x_batch, model_AT_rFGSM, trials=30, seed=230, is_report_loss_diff=True)

Mimicry*30: Attack effectiveness 0.000%.
Mimicry*30: Attack effectiveness 0.000%.
Mimicry*30: Attack effectiveness 33.333%.
Mimicry*30: Attack effectiveness 0.000%.
Mimicry*30: Attack effectiveness 10.526%.
Mimicry*30: Attack effectiveness 40.000%.
Mimicry*30: Attack effectiveness 37.500%.
Mimicry*30: Attack effectiveness 18.750%.
Mimicry*30: Attack effectiveness 12.500%.
Mimicry*30: Attack effectiveness 7.692%.
Mimicry*30: Attack effectiveness 23.529%.
Mimicry*30: Attack effectiveness 30.769%.
Mimicry*30: Attack effectiveness 21.429%.
Mimicry*30: Attack effectiveness 20.000%.
Mimicry*30: Attack effectiveness 0.000%.
Mimicry*30: Attack effectiveness 17.647%.
Mimicry*30: Attack effectiveness 36.842%.
Mimicry*30: Attack effectiveness 8.333%.
Mimicry*30: Attack effectiveness 12.500%.
Mimicry*30: Attack effectiveness 20.000%.
Mimicry*30: Attack effectiveness 22.222%.
Mimicry*30: Attack effectiveness 23.077%.
Mimicry*30: Attack effectiveness 11.111%.
Mimicry*30: Attack effectiveness 33.333%

In [ ]:
trials = 30
seed = 230
is_report_loss_diff = True

model_DNN.eval()

# Ensure trials do not exceed the length of ben_x
trials = min(trials, len(ben_x))

# Get the number of malicious samples
n_samples = len(malicious_samples)

if n_samples > 0:
    # Expand dimensions for efficient broadcasting
    malicious_samples_expanded = malicious_samples.unsqueeze(1).expand(-1, trials, -1)

    # Generate random indices for sampling from ben_x
    seed += 1
    torch.manual_seed(seed)
    indices = torch.randperm(len(ben_x), device=device)[:trials]
    trial_vectors_expanded = ben_x[indices].unsqueeze(0)

    # Perform the mimic attack
    pertbx = torch.clamp(malicious_samples_expanded + trial_vectors_expanded, min=0., max=1.)

    # Compute the loss and check if adversarial examples are successful
    loss, done = get_loss(pertbx.view(-1, pertbx.shape[-1]), torch.ones(n_samples * trials, 1, device=device), model_DNN)
    print(done)
    # Add maximum loss to successful attacks to differentiate
    max_v = loss.max()
    loss[done] += max_v

    # Reshape the loss and done tensors
    loss = loss.view(n_samples, trials)
    done = done.view(n_samples, trials)

    # Report attack effectiveness if required
    if is_report_loss_diff:
        n_done = torch.any(done, dim=-1).sum()
        print(n_done)
        print(f"Mimicry*{trials}: Attack effectiveness {n_done / n_samples * 100:.3f}%.")

    # Get the index of the maximum loss for each sample
    _, indices = loss.max(dim=-1)
    adv_x = pertbx[torch.arange(n_samples), indices]


tensor([False, False, False, False, False, False, False, False, False, False,
         True,  True, False, False, False,  True, False, False, False, False,
         True, False, False, False, False, False, False,  True, False, False,
         True, False, False,  True,  True, False,  True, False, False, False,
         True,  True, False,  True, False,  True, False, False,  True, False,
         True, False,  True,  True, False, False,  True,  True, False,  True,
        False, False, False, False,  True, False, False, False, False, False,
         True,  True, False, False, False,  True, False, False, False, False,
         True, False, False, False, False, False, False,  True, False, False,
        False, False, False,  True,  True, False,  True, False, False,  True,
         True,  True, False,  True, False,  True, False, False,  True, False,
         True, False,  True,  True, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, 

In [ ]:
adv_x.shape

torch.Size([12, 10000])

In [ ]:
# Example usage:
trials = 30
seed = 230
is_report_loss_diff = True

adv_x = mimicry(ben_x, malicious_samples, model_DNN, trials, seed, is_report_loss_diff, device="cpu")

Mimicry*30: Attack effectiveness 91.667%.


In [ ]:
outputs = model_AT_rFGSM(adv_x)
_, y_pred = torch.topk(outputs, k=1)

acc_ad_test = (y_pred == 1.).sum().item() / len(y_pred)
acc_ad_test

1.0

In [ ]:
def mimicry(ben_x, malicious_samples, model, seed, trials=1000, device="cpu"):


  model.eval()

  # Initialize counters
  successful_attacks = 0
  total_malicious_samples = 0

  trials = min(trials, len(ben_x))


  if len(malicious_samples) > 0:

    # Generate random indices outside the loop
    seed += 1
    torch.manual_seed(seed)
    indices = torch.randperm(len(ben_x), device=device)[:trials]
    trial_vectors = ben_x[indices]

    # Perform the mimic attack and update counters
    modified_x = torch.clamp(malicious_samples + trial_vectors, min=0., max=1.)
    _, done = get_loss(modified_x.view(-1, modified_x.shape[-1]), torch.ones(trials * malicious_samples.shape[0], 1, device=device), model)
    successful_attacks += (done.view(malicious_samples.shape[0], trials).sum(dim=1) > 0).sum().item()
    total_malicious_samples += malicious_samples.shape[0]

  # Calculate and print attack effectiveness
  attack_effectiveness = (successful_attacks / total_malicious_samples) * 100 if total_malicious_samples > 0 else 0
  print(f"Mimic attack effectiveness: {attack_effectiveness:.3f}%.")

  return attack_effectiveness  # Added return statement for clarity


In [ ]:
def mimic_attack_effectiveness_optimized(test_loader, model, seed, trials=1000, device="cuda:0"):
  """
  Calculates the effectiveness of the mimic attack on the given model.

  Args:
      test_loader: A PyTorch dataloader containing the test data.
      model: The PyTorch model to be attacked.
      seed: The random seed for reproducibility.
      trials: The number of random samples to use from the benign class (default: 1000).
      device: The device to use for computations (default: "cuda:0" if available, otherwise "cpu").

  Returns:
      The effectiveness of the mimic attack as a percentage (float).
  """

  torch.manual_seed(seed)
  model.eval()

  # Initialize counters
  successful_attacks = 0
  total_malicious_samples = 0

  # Pre-select benign samples for efficiency
  benign_samples = []
  for x_batch, y_batch in test_loader:
    benign_samples.append(x_batch[y_batch.squeeze() == 0])

  ben_x = torch.cat(benign_samples, dim=0).to(device)

  # Clear unnecessary variables
  del benign_samples

  trials = min(trials, len(ben_x))


  for x_batch, y_batch in test_loader:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    malicious_samples = x_batch[y_batch.squeeze() == 1]

    if len(malicious_samples) > 0:
      # Expand dimensions for efficient broadcasting
      malicious_samples_expanded = malicious_samples.unsqueeze(1).expand(-1, trials, -1)

      # Generate random indices outside the loop
      seed += 1
      torch.manual_seed(seed)
      indices = torch.randperm(len(ben_x), device=device)[:trials]
      trial_vectors_expanded = ben_x[indices].unsqueeze(0)

      # Perform the mimic attack and update counters
      modified_x = torch.clamp(malicious_samples_expanded + trial_vectors_expanded, min=0., max=1.)
      _, done = get_loss(modified_x.view(-1, modified_x.shape[-1]), torch.ones(trials * malicious_samples.shape[0], 1, device=device), model)
      successful_attacks += (done.view(malicious_samples.shape[0], trials).sum(dim=1) > 0).sum().item()
      total_malicious_samples += malicious_samples.shape[0]
      break
  # Calculate and print attack effectiveness
  attack_effectiveness = (successful_attacks / total_malicious_samples) * 100 if total_malicious_samples > 0 else 0
  print(f"Mimic attack effectiveness: {attack_effectiveness:.3f}%.")

  return attack_effectiveness  # Added return statement for clarity


In [ ]:
mimic_attack_effectiveness_optimized(test_loader, model_DNN , seed=230, trials=100, device=device)

Mimic attack effectiveness: 91.667%.


91.66666666666666